In [15]:
import copy
import warnings
import pandas as pd
import numpy as np
from model import Company, StockLog, IndexLog, PerformanceLog
from model import start_engine
from datetime import date, datetime

warnings.filterwarnings("ignore")

In [16]:
engine = start_engine()

In [17]:
def get_table(
    stock,                      # list of stocks 
    start = '2010-01-01',       # start date in 'YYYY-MM-DD' format
    finish = None,              # finish date in 'YYYY-MM-DD' format
    engine = engine,
    change = False,             # daily value change    
    change_pct = False,         # daily percentage change    
    rank = False,               # daily rank
    # observations
    opening = False,
    high = False,
    low = False,
    closing = False,
    adjusted = False,
    volume = False,
    # fundamentals
    pe_ratio = False,
    dp_ratio = False,
    pb_ratio = False):
    
    start = datetime.strptime(start, '%Y-%m-%d').date()
    if finish:
        finish = datetime.strptime(finish, '%Y-%m-%d').date()
    else:
        finish = datetime.today()
     
    company = Company().query().get(stock)
    stock_log = StockLog().query().filter((StockLog.company == company) &
                                           (StockLog.date >= start) &
                                           (StockLog.date <= finish))
    df = pd.read_sql(stock_log.statement, engine)
    df.sort_values(by='date', inplace=True)
    df.set_index('date', inplace=True)

    table = pd.DataFrame(index=df.index)
    
    if opening:
        table['opening'] = df['opening']
    if high:
        table['high'] = df['high']
    if low:
        table['low'] = df['low']
    if closing:
        table['closing'] = df['closing']
    if adjusted:
        table['adjusted'] = df['adjusted']        
    if pe_ratio:
        table['pe_ratio'] = df['pe_ratio']
    if dp_ratio:
        table['dp_ratio'] = df['dp_ratio']
    if pb_ratio:
        table['pb_ratio'] = df['pb_ratio']
    if change:
        table['change'] = df['closing'].diff() 
    if change_pct:
        table['change_pct'] = df['change_pct'] * 100
    if rank:
        table['rank'] = df['rank']      
        
    return table

In [18]:
stock = 'ABC'

In [19]:
get_table(stock, opening=1, high=1, low=1, closing=1, adjusted=1, pe_ratio=1, dp_ratio=1, pb_ratio=1, change=1, change_pct=1, rank=1).tail()

,opening,high,low,closing,adjusted,pe_ratio,dp_ratio,pb_ratio,change,change_pct,rank
date,,,,,,,,,,,
2019-07-22,4.36,4.46,4.29,4.46,4.46,0.157042,6.27803,2.33422,0.11,2.528740,5
2019-07-23,4.44,4.48,4.26,4.29,4.29,0.151056,6.52681,2.24525,-0.17,-3.811660,78
2019-07-24,4.30,4.36,4.27,4.33,4.33,0.152465,6.46651,2.26619,0.04,0.932401,39
2019-07-25,4.34,4.40,4.32,4.39,4.39,0.154577,6.37813,2.29759,0.06,1.385680,22
2019-07-26,4.37,4.38,4.30,4.34,4.34,0.152817,6.45161,2.27142,-0.05,-1.138950,65
